In [4]:
import numpy as np
import os
import time
import threading
import multiprocessing
from scipy.io import wavfile
from numba import cuda
from tqdm import tqdm

In [5]:


# Define CUDA Kernel for noise removal
@cuda.jit
def noise_removal_kernel(input_data, output_data, size):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    if idx < size:
        left = input_data[idx - 1] if idx > 0 else input_data[idx]
        right = input_data[idx + 1] if idx < size - 1 else input_data[idx]
        output_data[idx] = input_data[idx] - 0.5 * (left + right)

# Function to process a file sequentially
def process_sequential(input_path, output_path):
    sample_rate, data = wavfile.read(input_path)
    data = data.astype(np.float32)
    data /= np.max(np.abs(data))  # Normalize to range [-1, 1]
    
    # Apply noise removal (simplified sequential logic)
    processed_data = data - 0.5 * (np.roll(data, 1) + np.roll(data, -1))

    processed_data = (processed_data * 32767).astype(np.int16)  # Denormalize to int16
    wavfile.write(output_path, sample_rate, processed_data)

# Function to process a file using multi-threading
def process_threading(input_path, output_path):
    sample_rate, data = wavfile.read(input_path)
    data = data.astype(np.float32)
    data /= np.max(np.abs(data))  # Normalize to range [-1, 1]
    
    # Apply noise removal (simplified logic for threading)
    processed_data = data - 0.5 * (np.roll(data, 1) + np.roll(data, -1))

    processed_data = (processed_data * 32767).astype(np.int16)  # Denormalize to int16
    wavfile.write(output_path, sample_rate, processed_data)

# Function to process a file using multi-processing
def process_multiprocessing(input_path, output_path):
    sample_rate, data = wavfile.read(input_path)
    data = data.astype(np.float32)
    data /= np.max(np.abs(data))  # Normalize to range [-1, 1]
    
    # Apply noise removal (simplified logic for multiprocessing)
    processed_data = data - 0.5 * (np.roll(data, 1) + np.roll(data, -1))

    processed_data = (processed_data * 32767).astype(np.int16)  # Denormalize to int16
    wavfile.write(output_path, sample_rate, processed_data)

# Function to process a file using CUDA
def process_cuda(input_path, output_path):
    sample_rate, data = wavfile.read(input_path)
    data = data.astype(np.float32)
    data /= np.max(np.abs(data))  # Normalize to range [-1, 1]

    # Prepare CUDA parameters
    size = data.size
    input_gpu = cuda.to_device(data)
    output_gpu = cuda.device_array_like(data)
    threads_per_block = 256
    blocks_per_grid = (size + threads_per_block - 1) // threads_per_block

    # Apply CUDA noise removal
    noise_removal_kernel[blocks_per_grid, threads_per_block](input_gpu, output_gpu, size)
    cuda.synchronize()

    # Retrieve processed data and save
    processed_data = output_gpu.copy_to_host()
    processed_data = (processed_data * 32767).astype(np.int16)  # Denormalize to int16
    wavfile.write(output_path, sample_rate, processed_data)

# Function to process all files in a directory using sequential approach
def process_directory_sequential(input_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)
    files = [f for f in os.listdir(input_directory) if f.endswith(".wav")]
    for file in tqdm(files, desc="Processing audio files sequentially", unit="file", ncols=100):
        input_path = os.path.join(input_directory, file)
        output_path = os.path.join(output_directory, file)
        process_sequential(input_path, output_path)

# Function to process all files in a directory using multi-threaded approach
def process_directory_threading(input_directory, output_directory, num_threads):
    os.makedirs(output_directory, exist_ok=True)
    files = [f for f in os.listdir(input_directory) if f.endswith(".wav")]
    threads = []
    
    # Distribute files across threads
    files_per_thread = len(files) // num_threads
    for i in tqdm(range(num_threads), desc="Processing audio files with threading", unit="thread", ncols=100):
        start_idx = i * files_per_thread
        end_idx = (i + 1) * files_per_thread if i != num_threads - 1 else len(files)
        for file in files[start_idx:end_idx]:
            input_path = os.path.join(input_directory, file)
            output_path = os.path.join(output_directory, file)
            thread = threading.Thread(target=process_threading, args=(input_path, output_path))
            threads.append(thread)
            thread.start()

    for thread in threads:
        thread.join()

# Function to process all files in a directory using multi-processing approach
def process_directory_multiprocessing(input_directory, output_directory, num_processes):
    os.makedirs(output_directory, exist_ok=True)
    files = [f for f in os.listdir(input_directory) if f.endswith(".wav")]
    pool = multiprocessing.Pool(processes=num_processes)
    
    # Apply function to each file using pool of processes
    for file in tqdm(files, desc="Processing audio files with multiprocessing", unit="file", ncols=100):
        input_path = os.path.join(input_directory, file)
        output_path = os.path.join(output_directory, file)
        pool.apply_async(process_multiprocessing, args=(input_path, output_path))

    pool.close()
    pool.join()

# Function to process all files in a directory using CUDA approach
def process_directory_cuda(input_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)
    files = [f for f in os.listdir(input_directory) if f.endswith(".wav")]
    for file in tqdm(files, desc="Processing audio files with CUDA", unit="file", ncols=100):
        input_path = os.path.join(input_directory, file)
        output_path = os.path.join(output_directory, file)
        process_cuda(input_path, output_path)

# Function to calculate and compare the metrics for all approaches
def calculate_metrics(sequential_time, parallel_time_threading, parallel_time_multiprocessing, cuda_time, total_files, num_cores):
    throughput_sequential = total_files / sequential_time
    throughput_threading = total_files / parallel_time_threading
    throughput_multiprocessing = total_files / parallel_time_multiprocessing
    throughput_cuda = total_files / cuda_time

    latency_sequential = sequential_time / total_files
    latency_threading = parallel_time_threading / total_files
    latency_multiprocessing = parallel_time_multiprocessing / total_files
    latency_cuda = cuda_time / total_files

    speedup_threading = sequential_time / parallel_time_threading
    efficiency_threading = (speedup_threading / num_cores) * 100

    speedup_multiprocessing = sequential_time / parallel_time_multiprocessing
    efficiency_multiprocessing = (speedup_multiprocessing / num_cores) * 100

    speedup_cuda = sequential_time / cuda_time
    efficiency_cuda = (speedup_cuda / num_cores) * 100

    print(f"Sequential Time: {sequential_time:.2f} seconds")
    print(f"Threading Time: {parallel_time_threading:.2f} seconds")
    print(f"Multiprocessing Time: {parallel_time_multiprocessing:.2f} seconds")
    print(f"CUDA Time: {cuda_time:.2f} seconds")

    print(f"Throughput (Sequential): {throughput_sequential:.2f} tasks/s")
    print(f"Throughput (Threading): {throughput_threading:.2f} tasks/s")
    print(f"Throughput (Multiprocessing): {throughput_multiprocessing:.2f} tasks/s")
    print(f"Throughput (CUDA): {throughput_cuda:.2f} tasks/s")

    print(f"Latency (Sequential): {latency_sequential:.4f} seconds/task")
    print(f"Latency (Threading): {latency_threading:.4f} seconds/task")
    print(f"Latency (Multiprocessing): {latency_multiprocessing:.4f} seconds/task")
    print(f"Latency (CUDA): {latency_cuda:.4f} seconds/task")

    print(f"Speedup (Threading): {speedup_threading:.2f}")
    print(f"Efficiency (Threading): {efficiency_threading:.2f}%")

    print(f"Speedup (Multiprocessing): {speedup_multiprocessing:.2f}")
    print(f"Efficiency (Multiprocessing): {efficiency_multiprocessing:.2f}%")

    print(f"Speedup (CUDA): {speedup_cuda:.2f}")
    print(f"Efficiency (CUDA): {efficiency_cuda:.2f}%")



In [6]:
# Main execution
if __name__ == "__main__":
    input_dir = "/home/mahmoud/Desktop/Projects/parallelproject/segmenteddataset/segmentedseq_multiprocessing"
    output_dir_sequential = "/home/mahmoud/Desktop/Projects/parallelproject/noiseremoveddataset/noise_sequential"
    output_dir_threading = "/home/mahmoud/Desktop/Projects/parallelproject/noiseremoveddataset/noise_threaded"
    output_dir_multiprocessing = "/home/mahmoud/Desktop/Projects/parallelproject/noiseremoveddataset/noisemultiprocess"
    output_dir_cuda = "/home/mahmoud/Desktop/Projects/parallelproject/noiseremoveddataset/noisebycuda"

    # Get the number of CPU cores
    num_cores = os.cpu_count()

    # Measure execution time for sequential processing
    start_time = time.time()
    process_directory_sequential(input_dir, output_dir_sequential)
    sequential_time = time.time() - start_time

    # Measure execution time for threading
    start_time = time.time()
    process_directory_threading(input_dir, output_dir_threading, num_threads=num_cores)
    parallel_time_threading = time.time() - start_time

    # Measure execution time for multiprocessing
    start_time = time.time()
    process_directory_multiprocessing(input_dir, output_dir_multiprocessing, num_processes=num_cores)
    parallel_time_multiprocessing = time.time() - start_time

    # Measure execution time for CUDA
    start_time = time.time()
    process_directory_cuda(input_dir, output_dir_cuda)
    cuda_time = time.time() - start_time

    # Get the total number of files processed
    total_files = len([f for f in os.listdir(input_dir) if f.endswith(".wav")])

    if total_files == 0:
        print("No .wav files found in the input directory.")
    else:
        # Calculate metrics
        calculate_metrics(sequential_time, parallel_time_threading, parallel_time_multiprocessing, cuda_time, total_files, num_cores)

Processing audio files sequentially:   0%|                                | 0/183 [00:00<?, ?file/s]

Processing audio files with CUDA: 100%|█████████████████████████| 183/183 [00:30<00:00,  6.01file/s]

Sequential Time: 40.65 seconds
Threading Time: 13.18 seconds
Multiprocessing Time: 12.66 seconds
CUDA Time: 30.48 seconds
Throughput (Sequential): 4.50 tasks/s
Throughput (Threading): 13.89 tasks/s
Throughput (Multiprocessing): 14.46 tasks/s
Throughput (CUDA): 6.00 tasks/s
Latency (Sequential): 0.2221 seconds/task
Latency (Threading): 0.0720 seconds/task
Latency (Multiprocessing): 0.0692 seconds/task
Latency (CUDA): 0.1665 seconds/task
Speedup (Threading): 3.08
Efficiency (Threading): 15.42%
Speedup (Multiprocessing): 3.21
Efficiency (Multiprocessing): 16.06%
Speedup (CUDA): 1.33
Efficiency (CUDA): 6.67%
